
# Modelos Autoregresivos

## US population dataset
Población mensual de E.U estimada en miles de personas, desde Enero 2011 a Diciembre 2018 (96 datos, 8 años).

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

# Load specific forecasting tools
from statsmodels.tsa.ar_model import AR,ARResults
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
# Load the U.S. Population dataset
df = pd.read_csv('./uspopulation.csv',index_col='DATE',parse_dates=True)
df.index.freq = 'MS'

In [ ]:
df.head()

## Gráfico de la serie

In [ ]:
title='U.S. Monthly Population Estimates'
ylabel='Pop. Est. (thousands)'

ax = df['PopEst'].plot(figsize=(12,5),title=title);
ax.autoscale(axis='x',tight=True)
ax.set(ylabel=ylabel);

## Datos entrenamiento/prueba


In [ ]:
len(df)

In [ ]:
train = df.iloc[:84]
test = df.iloc[84:]

## Modelo AR(1)

In [ ]:
# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

In [ ]:
model = AutoReg(train['PopEst'], lags=1)
AR1fit = model.fit()
print(f'Lag: {AR1fit.model.ar_lags}')
print(f'Coefficients:\n{AR1fit.params}')

In [ ]:
AR1fit.summary()

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predictions1 = AR1fit.predict(start=start, end=end, dynamic=False).rename('AR(1) Predictions')

In [ ]:
predictions1

In [ ]:
# Comparando predicciones con valores reales
for i in range(len(predictions1)):
    print(f"predicted={predictions1[i]:<11.10}, expected={test['PopEst'][i]}")

In [ ]:
test['PopEst'].plot(legend=True)
predictions1.plot(legend=True,figsize=(12,6));

## Modelo AR(2)

In [ ]:
model = AutoReg(train['PopEst'], lags=2)
AR2fit = model.fit()
#AR2fit = model.fit(maxlag=2,method='mle')
print(f'Lag: {AR2fit.model.ar_lags}')
print(f'Coefficients:\n{AR2fit.params}')

In [ ]:
AR2fit.summary()

In [ ]:
start=len(train)
end=len(train)+len(test)-1
predictions2 = AR2fit.predict(start=start, end=end, dynamic=False).rename('AR(2) Predictions')

In [ ]:
test['PopEst'].plot(legend=True)
predictions1.plot(legend=True)
predictions2.plot(legend=True,figsize=(12,6));

In [ ]:
print(f'Error AIC modelo AR1:{AR1fit.aic}')
print(f'Error AIC modelo AR2:{AR2fit.aic}')

El error en el segundo modelo es menor, luego es mejor modelo el AR(2), como opodemos apreciar en la gráfica

## Modelo AR(p) donde p es un valor estudiado como el mejor


In [ ]:
model = AutoReg(train['PopEst'], lags=8)
AR8fit = model.fit()

print(f'Lag: {AR8fit.model.ar_lags}')
print(f'Coefficients:\n{AR8fit.params}')

In [ ]:
start = len(train)
end = len(train)+len(test)-1
rename = f'AR(8) Predictions'

predictions8 = AR8fit.predict(start=start,end=end,dynamic=False).rename(rename)

In [ ]:
test['PopEst'].plot(legend=True)
predictions1.plot(legend=True)
predictions2.plot(legend=True)
predictions8.plot(legend=True,figsize=(12,6));

In [ ]:
AR8fit.summary()

## Evaluar el modelo
Error cuadrático medio (*Mean Squared Error*, MSE) y raíz del error cuadrático medio (RMSE)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

labels = ['AR(1)','AR(2)','AR(8)']
preds = [predictions1, predictions2, predictions8]  # these are variables, not strings!

for i in range(3):
    mse = mean_squared_error(test['PopEst'], preds[i])
    print(f'{labels[i]} Error MSE: {mse:11.10}')
    rmse = root_mean_squared_error(test['PopEst'], preds[i])
    print(f'{labels[i]} Error RMSE: {rmse:11.10}')

AIC (Akaike Information Criteriom)

In [ ]:
modls = [AR1fit,AR2fit,AR8fit]

for i in range(3):
    print(f'{labels[i]} AIC: {modls[i].aic:6.5}')

**AIC** y **BIC** son dos criterios que miden la bondad de ajuste de un modelo estadístico, teniendo en cuenta tanto la complejidad como la precisión del modelo. AIC significa Akaike Information Criterion, y BIC significa Bayesian Information Criterion. Ambos criterios se basan en la función de verosimilitud, que cuantifica qué tan bien el modelo se ajusta a los datos y penaliza al modelo por tener más parámetros, lo que aumenta el riesgo de sobreajuste. Cuanto más bajo sea el AIC o BIC, mejor será el modelo


[Choosing the Best Model: A Friendly Guide to AIC and BIC](https://medium.com/@jshaik2452/choosing-the-best-model-a-friendly-guide-to-aic-and-bic-af220b33255f)

[Common metrics for Time Series Analysis](https://joydeep31415.medium.com/common-metrics-for-time-series-analysis-f3ca4b29fe42)

## Predicciones

In [ ]:
# First, retrain the model on the full dataset
model = AutoReg(df['PopEst'], lags=8)

# Next, fit the model
ARfit = model.fit()

# Make predictions
fcast = ARfit.predict(start=len(df), end=len(df)+12, dynamic=False).rename('Forecast')

# Plot the results
df['PopEst'].plot(legend=True)
fcast.plot(legend=True,figsize=(12,6));

### Preguntas

+ ¿Como de buena es la predicción? ¿Como podemos valorarla objetivamente?
+ ¿Como sabemos qué **cantidad de lags** son necesarios para una buena predicción?

In [ ]:
import statsmodels.graphics.tsaplots as sgt
import matplotlib.pyplot as plt

sgt.plot_acf(df, lags = 40, zero = False)
plt.title("Autocorrelación", size = 12)
plt.show()

In [ ]:
import statsmodels.graphics.tsaplots as sgt
import matplotlib.pyplot as plt

sgt.plot_pacf(df, lags = 40, zero = False)
plt.title("Autocorrelación Parcial", size = 12)
plt.show()

# EOF (End Of File)